In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Function to read the data
def read_data(filename):
    data = pd.read_csv(filename, header = None)
    features = data.iloc[:, :-1].values
    labels = data.iloc[:, -1].values - 1  # Shift the class labels to 0-25
    return features, labels

# Read the data
data_url = "https://raw.githubusercontent.com/UMBInal/data/main/data.csv"
x, y = read_data(data_url)

In [3]:
x[0]

array([0.01176471, 0.01176471, 0.97647059, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [4]:
y[0]

0

In [5]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [6]:
x_train[0]

array([0.  , 0.  , 0.  , 0.08, 0.76, 0.12, 0.  , 0.  , 0.04, 0.  ])

In [7]:
len(x_train)

25669

In [8]:
x_train.shape

(25669, 10)

In [9]:
y_train.shape

(25669,)

In [10]:
# Reshape the data
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [11]:
x_train.shape

(25669, 10, 1)

In [12]:
# Determine the input shape and number of classes
input_shape = (x_train.shape[1], 1)
num_classes = len(np.unique(y))

In [13]:
num_classes

26

In [14]:
input_shape

(10, 1)

In [15]:
# Transformer Encoder function
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed-forward network
    x = layers.Dense(ff_dim, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(128, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [16]:
# Transformer Decoder function
def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(res, enc_outputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res

    # Feed-forward network
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(128, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    
    x = layers.Dense(inputs.shape[-1], activation="softmax")(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [17]:
# Build the model
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Decoder
    decoder_input = keras.Input(shape=input_shape)
    dec = decoder_input
    for _ in range(num_transformer_blocks):
        dec = transformer_decoder(dec, x, head_size, num_heads, ff_dim, dropout)

    x = layers.Flatten()(x)
    dec = layers.Flatten()(dec)
    x = layers.Concatenate()([x, dec])

    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model([inputs, decoder_input], outputs)
    return model

In [18]:
# Define the hyperparameters
head_size = 2
num_heads = 8
ff_dim = 64
num_transformer_blocks = 6
mlp_units = [64]
dropout = 0.1
mlp_dropout = 0.1

In [19]:
# Build and compile the model
model = build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout, mlp_dropout)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# Train the model
model.fit([x_train, x_train], y_train, batch_size=64, epochs=200, validation_split=0.3)


Epoch 1/200
281/281 [==============================] - 24s 33ms/step - loss: 2.0827 - accuracy: 0.4392 - val_loss: 1.6181 - val_accuracy: 0.5234
Epoch 2/200
281/281 [==============================] - 8s 28ms/step - loss: 1.5238 - accuracy: 0.5348 - val_loss: 1.4175 - val_accuracy: 0.5612
Epoch 3/200
281/281 [==============================] - 8s 28ms/step - loss: 1.4039 - accuracy: 0.5549 - val_loss: 1.3401 - val_accuracy: 0.5777
Epoch 4/200
281/281 [==============================] - 8s 28ms/step - loss: 1.3459 - accuracy: 0.5689 - val_loss: 1.2955 - val_accuracy: 0.5884
Epoch 5/200
281/281 [==============================] - 8s 28ms/step - loss: 1.3098 - accuracy: 0.5743 - val_loss: 1.2663 - val_accuracy: 0.6020
Epoch 6/200
281/281 [==============================] - 8s 28ms/step - loss: 1.2882 - accuracy: 0.5774 - val_loss: 1.2425 - val_accuracy: 0.6017
Epoch 7/200
281/281 [==============================] - 8s 29ms/step - loss: 1.2658 - accuracy: 0.5874 - val_loss: 1.2264 - val_accuracy

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([x_test, x_test], y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")